# Best Cities for University Students

#### Introduction to Business Problem
-----

This notebook is going to study and analyse the cities that are ranked best for university students. For many students and agencies, it is quiet troublesome to analyse which city is best for them to pursue a mayor not only on the degrees offered by the university's programs, but to the livestyle the university students want to have around them. Therefore, it is of mayor interest to cluster the best university cities around the globe in order to provide an overview and a list of similar options to students and agencies.  

Using the list of cities created by the __QS World University Rankings__, which can be found at [QS Best Student Cities](https://www.topuniversities.com/city-rankings/2019), we are going to observe similar cities based on the kind of businesses and venues they have around __10km__ from their _central business district_. This area of each city gives similar chances to see what kind of venues are agglomerated within the business center which can be helpful to most students that want to work and live in the same city they study their degree.

On another note, the QS rankings were derived from scores on in eight out of 12 categories. Four categories are mandatory, while institutions must choose the remaining four optional categories:
- Teaching
- Employability
- Research
- Internationalization
- Facilities
- Online/Distance Learning
- Arts & Culture
- Innovation
- Inclusiveness
- Social Responsibility
- Subject Ranking
- Program Strength

### Data for Business Problem
-----

In [2]:
# Installing necessary packages for webscraping and creation of map visualizations
!conda install beautifulsoup4
!conda install lxml
!conda install html5lib
!conda install requests
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB
    beautifulsoup4-4.8.2       |           py36_0         161 KB
    ------------------------------------------------------------
                                           Total:         294 KB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1 --> 4.8.2-py36_0
    ca-certificates: 2019.11.27-0 --> 2020.1.1-0  


ca-certificates-2020 | 132 KB    | ##################################### | 100% 
beautifulsoup4-4.8.2 | 161 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environme

In [3]:
# Loading necessary packages for data manipulation and webscraping
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [25]:
# The code was removed by Watson Studio for sharing.

,rank,city,country,lat,lon
0,1,London,United Kingdom,17.983333,-88.433333
1,2,Tokyo,Japan,35.685000,139.751389
2,3,Melbourne,Australia,-37.813938,144.963425
3,4,Munich,Germany,48.150000,11.583300
4,5,Berlin,Germany,-18.583333,-68.233333


__Foursquare Credentials and Version__

In [34]:
CLIENT_ID = 'DRF0SKJCMYWGQ5X0OO1KZJGFQL0BSF4WS53CUU53RJAVMMSD'
CLIENT_SECRET = 'DRGDZ0HCIGEG1CGWG3D1FYDREMR10XO3NF50FXYSOSNLU5BF'
VERSION = '20180605'

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use the function define above to collect the venue data for only the _10km_ within each __City__:

In [41]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 10000 # define radius

cities_venues = getNearbyVenues(names=qs_ranking['city'],
                                   latitudes=qs_ranking['lat'],
                                   longitudes=qs_ranking['lon']
                                  )

London
Tokyo
Melbourne
Munich
Berlin
Montreal
Paris
Zurich
Sydney
Hong Kong
Seoul
Toronto
Boston
Vienna
Edinburgh
Vancouver
Taipei
Kyoto
New York
Singapore
Barcelona
Brisbane
Canberra
Amsterdam
Los Angeles
Adelaide
Auckland
Madrid
Kuala Lumpur
Manchester
Buenos Aires
Beijing
Shanghai
Moscow
Prague
San Francisco
Dublin
Stockholm
Glasgow
Milan
Chicago
Perth
Brussels
Budapest
Ottawa
Copenhagen
Coventry
Nottingham
Birmingham
Lyon
Warsaw
Lisbon
Mexico
Santiago
Gothenburg
Helsinki
Atlanta
Newcastle Upon Tyne
Bangkok
Brno
Kubai
Philadelphia
Washington
Aberdeen
Christchurch
Saint Petersburg
Pittsburgh
Hsinchu
Stuttgart
Rome
Istanbul
Athens
Tomsk
Toulouse
Valencia
San Diego
Sao Paulo
Johannesburg
Graz
Cape Town
Bangalore
Bogota
Brighton
Gold Coast
Manila
Mumbai
Baltimore
Abu Dhabi
Ankara
Daejeon
Monterrey
Novosibirsk
Houston
Miami
Nagoya
Vilnius
Bristol
Lima
Nanjing
Montpellier
Beirut
Rio De Janeiro
Sheffield
Krakow
Wuhan
Liverpool
Cairo
Wroclaw
Jakarta
Riga
Islamabad
Almaty
Delhi
Dallas
Chenna

Encode each __City__ with its information on venues for clustering:

In [42]:
# one hot encoding
cities_onehot = pd.get_dummies(cities_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cities_onehot['City'] = cities_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [cities_onehot.columns[-1]] + list(cities_onehot.columns[:-1])
cities_onehot = cities_onehot[fixed_columns]

cities_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the rows by __City__ and by taking the mean of frequency of each category of venue:

In [45]:
cities_grouped = cities_onehot.groupby('City').mean().reset_index()
cities_grouped

,City,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,Aberdeen,0.000000,0.0,0.0,0.00,0.00,0.010000,0.02,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.01,0.0,0.000000,0.00,0.00,0.00
1,Abu Dhabi,0.000000,0.0,0.0,0.00,0.01,0.000000,0.00,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.01,0.00
2,Adelaide,0.010000,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.010000,...,0.00,0.00,0.01,0.01,0.00,0.0,0.000000,0.00,0.00,0.00
3,Almaty,0.000000,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.010000,...,0.00,0.00,0.00,0.01,0.00,0.0,0.000000,0.00,0.01,0.00
4,Amsterdam,0.013158,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.013158,...,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.00,0.00
5,Ankara,0.000000,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.00,0.00
6,Athens,0.000000,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,...,0.00,0.03,0.03,0.00,0.00,0.0,0.000000,0.00,0.00,0.00
7,Atlanta,0.000000,0.0,0.0,0.00,0.00,0.142857,0.00,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.00,0.00
8,Auckland,0.000000,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.00,0.00
9,Bangalore,0.000000,0.0,0.0,0.01,0.00,0.000000,0.00,0.000000,0.010000,...,0.00,0.00,0.00,0.00,0.00,0.0,0.010000,0.00,0.00,0.00


Let's define a function in order to obtain the _top 10_ kind of venues of each __City__:

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [83]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = cities_grouped['City']

for ind in np.arange(cities_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cities_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aberdeen,Beach,Caribbean Restaurant,Bar,Resort,Shopping Mall,Grocery Store,Hotel,Fast Food Restaurant,Pizza Place,Sandwich Place
1,Abu Dhabi,Café,Coffee Shop,Hotel,Beach,Middle Eastern Restaurant,Bakery,Turkish Restaurant,Restaurant,Gym / Fitness Center,Australian Restaurant
2,Adelaide,Café,Coffee Shop,Hotel,Asian Restaurant,Australian Restaurant,Park,Pub,Garden,Sushi Restaurant,Mexican Restaurant
3,Almaty,Hotel,Coffee Shop,Café,Park,Steakhouse,Gym / Fitness Center,Multiplex,Bar,Turkish Restaurant,Pub
4,Amsterdam,Restaurant,Harbor / Marina,Burger Joint,Racetrack,BBQ Joint,Theme Park,Park,Lake,Golf Course,Cajun / Creole Restaurant


## Clustering of Neighborhoods

Define _10_ diferent clusters of __Cities__:

In [84]:
# set number of clusters
kclusters = 10

cities_grouped_clustering = cities_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cities_grouped_clustering)

# add clustering labels
cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cities_merged = qs_ranking
cities_merged.rename(columns={'city':'City'}, inplace=True)

# merge cities_grouped with qs_ranking to add latitude/longitude for each city
cities_merged = cities_merged.join(cities_venues_sorted.set_index('City'), on='City')
cities_merged['Cluster Labels'].fillna(10, inplace = True) 
cities_merged.round({"Cluster Labels":0})

cities_merged.head(120) # check the last columns!

,rank,City,country,lat,lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,London,United Kingdom,17.983333,-88.433333,8.0,Restaurant,Lake,Food Service,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop
1,2,Tokyo,Japan,35.685000,139.751389,8.0,Hotel,Garden,Coffee Shop,Chinese Restaurant,Ramen Restaurant,Wagashi Place,Tonkatsu Restaurant,Art Museum,Kaiseki Restaurant,Japanese Curry Restaurant
2,3,Melbourne,Australia,-37.813938,144.963425,8.0,Café,Coffee Shop,Park,Theater,Bar,Cocktail Bar,Ice Cream Shop,Japanese Restaurant,Wine Bar,Whisky Bar
3,4,Munich,Germany,48.150000,11.583300,8.0,Café,Plaza,German Restaurant,Hotel,Italian Restaurant,Church,Cocktail Bar,Gourmet Shop,Art Museum,Beer Garden
4,5,Berlin,Germany,-18.583333,-68.233333,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Montreal,Canada,-7.450000,-72.466667,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Paris,France,-13.550000,-62.483333,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Zurich,Switzerland,47.366667,8.550000,8.0,Bar,Swiss Restaurant,Plaza,Hotel,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Park,Cocktail Bar,Restaurant
8,9,Sydney,Australia,-33.861481,151.205475,8.0,Café,Park,Theater,Scenic Lookout,Coffee Shop,Bakery,Ice Cream Shop,Pub,Thai Restaurant,Hotel
9,10,Hong Kong,Hong Kong,22.283333,114.150000,8.0,Hotel,Coffee Shop,Gym / Fitness Center,Japanese Restaurant,Italian Restaurant,Cocktail Bar,Yoga Studio,Scenic Lookout,Park,Café


### Visualize the Clusters

In [86]:
# create map of Toronto using latitude and longitude values
world_map = folium.Map(location=[0, 0], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
markers_colors = []
for lat, lon, poi, cluster in zip(cities_merged['lat'], cities_merged['lon'], cities_merged['City'], cities_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1-1],
        fill=True,
        fill_color=rainbow[2-1],
        fill_opacity=0.7).add_to(world_map)
    
world_map